In [1]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [2]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}

val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

val dataPath= "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

// Load and parse the data file, converting it to a DataFrame.
val raw_sentiment = spark.read
    .format("csv")
    .option("header",false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("(case when target=4 then 1 else 0 end) as label","trim(tweet) as tweet")

raw_sentiment.groupBy($"label").count.show
raw_sentiment.where("label=0").show(5,150)
raw_sentiment.where("label=1").show(5,150)

+-----+------+
|label| count|
+-----+------+
|    1|800000|
|    0|800000|
+-----+------+

+-----+-------------------------------------------------------------------------------------------------------------------+
|label|                                                                                                              tweet|
+-----+-------------------------------------------------------------------------------------------------------------------+
|    0|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D|
|    0|    is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!|
|    0|                          @Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds|
|    0|                                                                     my whole body feels itchy and like its on fire|
|    0|     @nationwideclass no, it's not

dataSchema = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
dataPath = /home/jovyan/data/training.1600000.processed.noemoticon.csv
raw_sentiment = [label: int, tweet: string]


[label: int, tweet: string]

In [13]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}

// Без токенайзера в рандомном лесе тоже, как я понял, не обойтись...
val tokenizer = new Tokenizer()
    .setInputCol("tweet")
    .setOutputCol("words")

// Как не обойтись и без хэш-формирования фичей, т.е. индексатору фичей нужны на входе фичи, а не слова.
// Иными словами, тут мы выделяем из текста фичи
val hashingTF = new HashingTF()
    .setNumFeatures(1000)
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features")

// Запускаем индексацию меток по всему датасету
// Как я понял, он требует весь raw_sentiment, чтобы проиндексировать все возможные метки.
val labelIndexer = new StringIndexer()
  .setInputCol("label")
  .setOutputCol("indexedLabel")
  .fit(raw_sentiment)

// Индексируем полученные фичи, выделяя их категории
// Как я понял, это некоторые группы предопределенных значений вроде
//     фича1:(открыто, закрыто), фича2:(короткий, средний, длинный), фича3:(хороший, плохой, никакой) и т.д...
val featureIndexer = new VectorIndexer()
  .setInputCol("features")
  .setOutputCol("indexedFeatures")
  .setMaxCategories(4)

// Собственно, метод тренировки типа "случайный лес"
val rf = new RandomForestClassifier()
  .setLabelCol("indexedLabel")
  .setFeaturesCol("indexedFeatures")
  .setNumTrees(10)

// Восстанавливаем читаемые значения меток из какого-то внутреннего формата, вектора...
val labelConverter = new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("predictedLabel")
  .setLabels(labelIndexer.labels)

// Объединяем все это дело в пайплайн...
val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, labelIndexer, featureIndexer, rf, labelConverter))

tokenizer = tok_930362eb7679
hashingTF = hashingTF_899fa58d4332
labelIndexer = strIdx_40fd7cef6c8f
featureIndexer = vecIdx_99bb16a9b424
rf = rfc_cdb1e3df91b9


labelConverter: org.apache.spark.ml.feature.Ind...


rfc_cdb1e3df91b9

In [ ]:
// Будем тренировать не на 70%, как предлагает документация, а на 100%, как на лекции )
// Split the data into training and test sets (30% held out for testing).
// val Array(trainingData, testData) = raw_sentiment.randomSplit(Array(0.7, 0.3))

In [14]:
// Тренируем модель, попутно запуская в этом же пайплайне необходимые индексирования
val model = pipeline.fit(raw_sentiment)

model = pipeline_8c796ec229fd


pipeline_8c796ec229fd

In [7]:
// Путь к модели
val modelPath = "/home/jovyan/models/spark-ml-model"

modelPath = /home/jovyan/models/spark-ml-model


/home/jovyan/models/spark-ml-model

In [8]:
// Записываем модель для последующего применения в apply_model
model.write.overwrite().save(modelPath)

In [9]:
// Считываем модель для проверки
val sameModel = PipelineModel.load(modelPath)

sameModel = pipeline_e3869da6fdc9


pipeline_e3869da6fdc9

In [10]:
// Применяем модель и получаем соотв. датасет с предсказаниями
val predictionsDF = sameModel.transform(raw_sentiment)

predictionsDF = [label: int, tweet: string ... 8 more fields]


[label: int, tweet: string ... 8 more fields]

In [20]:
// Смотрим на полученный датасет в целом...
predictionsDF.show(5,15)

+-----+---------------+---------------+---------------+------------+---------------+---------------+---------------+----------+--------------+
|label|          tweet|          words|       features|indexedLabel|indexedFeatures|  rawPrediction|    probability|prediction|predictedLabel|
+-----+---------------+---------------+---------------+------------+---------------+---------------+---------------+----------+--------------+
|    0|@switchfoot ...|[@switchfoot...|(1000,[7,14,...|         0.0|(1000,[7,14,...|[8.800332775...|[0.440016638...|       1.0|             1|
|    0|is upset tha...|[is, upset, ...|(1000,[170,1...|         0.0|(1000,[170,1...|[10.69590843...|[0.534795421...|       0.0|             0|
|    0|@Kenichan I ...|[@kenichan, ...|(1000,[10,36...|         0.0|(1000,[10,36...|[10.98117528...|[0.549058764...|       0.0|             0|
|    0|my whole bod...|[my, whole, ...|(1000,[82,19...|         0.0|(1000,[82,19...|[10.47277023...|[0.523638511...|       0.0|             0|

In [21]:
// Смотрим подробнее на результат применения модели к заранее положит. и отрицат. твитам, т.е. грубо визуально тестируем модель
predictionsDF.where("label=0").limit(10).union(predictionsDF.where("label=1").limit(10)).select("predictedLabel", "label", "tweet", "words", "features").show(20,40)

+--------------+-----+----------------------------------------+----------------------------------------+----------------------------------------+
|predictedLabel|label|                                   tweet|                                   words|                                features|
+--------------+-----+----------------------------------------+----------------------------------------+----------------------------------------+
|             1|    0|@switchfoot http://twitpic.com/2y1zl ...|[@switchfoot, http://twitpic.com/2y1z...|(1000,[7,14,21,54,91,170,220,246,311,...|
|             0|    0|is upset that he can't update his Fac...|[is, upset, that, he, can't, update, ...|(1000,[170,193,223,248,281,333,343,37...|
|             0|    0|@Kenichan I dived many times for the ...|[@kenichan, i, dived, many, times, fo...|(1000,[10,36,77,188,207,248,329,338,3...|
|             0|    0|my whole body feels itchy and like it...|[my, whole, body, feels, itchy, and, ...|(1000,[82,191,296,33

In [24]:
predictionsDF.selectExpr("probability").show(5,150)

+----------------------------------------+
|                             probability|
+----------------------------------------+
| [0.4400166387510603,0.5599833612489398]|
|[0.5347954217012693,0.46520457829873074]|
|[0.5490587643880294,0.45094123561197075]|
|[0.5236385117783351,0.47636148822166496]|
| [0.4781623303690328,0.5218376696309672]|
+----------------------------------------+
only showing top 5 rows



lastException: Throwable = null


In [15]:
//Сохраняем в файл, чтобы помотреть детальнее...
predictionsDF.where("label=0").limit(15).union(predictionsDF.where("label=1").limit(15))
   .selectExpr(
       "label","tweet","cast (words as string)","cast (features as string)","cast (rawPrediction as string)","cast (probability as string)","prediction"
   )
   .coalesce(1)
   .write.format("csv")
   .mode("overwrite")
   .option("header", "true")
   .save("/home/jovyan/work/predictionsDF.csv")

In [16]:
// Смотрим на предсказания, выводя их в читабельном виде через udf...

import org.apache.spark.sql.functions._

val getProbability =
    udf(
        (prediction: org.apache.spark.ml.linalg.Vector) =>
        {
            BigDecimal(prediction(0)).setScale(2, BigDecimal.RoundingMode.HALF_UP)+
            " / "+
            BigDecimal(prediction(1)).setScale(2, BigDecimal.RoundingMode.HALF_UP)
        }
    )

predictionsDF.where("label=0").limit(10).union(predictionsDF.where("label=1").limit(10))
    .select(getProbability($"probability").alias("clean_probability_0_1"),$"label").show(20,100)

+---------------------+-----+
|clean_probability_0_1|label|
+---------------------+-----+
|          0.44 / 0.56|    0|
|          0.53 / 0.47|    0|
|          0.55 / 0.45|    0|
|          0.52 / 0.48|    0|
|          0.48 / 0.52|    0|
|          0.49 / 0.51|    0|
|          0.48 / 0.52|    0|
|          0.47 / 0.53|    0|
|          0.51 / 0.49|    0|
|          0.48 / 0.52|    0|
|          0.47 / 0.53|    1|
|          0.54 / 0.46|    1|
|          0.44 / 0.56|    1|
|          0.47 / 0.53|    1|
|          0.48 / 0.52|    1|
|          0.48 / 0.52|    1|
|          0.48 / 0.52|    1|
|          0.43 / 0.57|    1|
|          0.48 / 0.52|    1|
|          0.50 / 0.50|    1|
+---------------------+-----+



getProbability = UserDefinedFunction(<function1>,StringType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,StringType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))